In [ ]:
"""
Author: Vineet Madan
Purpose: Lab Assignment 2 Part 1 of CS529 Applied Artificial Intelligence
Date: 8 October 2019
"""

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import json
import random
from sklearn.decomposition import TruncatedSVD
np.random.seed(0)

In [ ]:
#import the datasets (in csv) as pd dataframes
mdf = pd.read_csv("./ml-latest-small/movies.csv")
rdf = pd.read_csv("./ml-latest-small/ratings.csv")

In [ ]:
"""
This cell contains all the basic data handling functions

Note-->The new movie ids are indexed from zero and not one...this dict maps the old movie ids to to the new ones
"""
#give new id(s) to the movies...maps old ids to new ones
new_movie_ids = {}
old_movie_ids_from_new = {}

for index, row in mdf.iterrows():
    new_movie_ids[int(row['movieId'])] = index
    old_movie_ids_from_new[index] = int(row['movieId'])

# function to get new movie id from an old movie id
def get_new_movie_id_from_old(old_movie_id):
    return new_movie_ids[old_movie_id]
#function to get an old movie id from a new one
def get_old_movie_id_from_new(new_movie_id):
    return old_movie_ids_from_new[new_movie_id]

#function to get name of a movie using its new_movie_id
def get_movie_name_by_new_id(new_movie_id):
    for index, row in mdf.iterrows():
        if get_new_movie_id_from_old(row['movieId'])==new_movie_id:
            return row['title']
    return None

#function to get name of a movie using its old_movie_id
def get_movie_name_by_old_id(old_movie_id):
    for index, row in mdf.iterrows():
        if row['movieId']==old_movie_id:
            return row['title']
    return None

#function to create the rating matrix
def create_the_rating_matrix(movies_df, num_movies, num_users):
    full_ratings_mat = np.empty(shape=(num_users+1, num_movies))
    full_ratings_mat.fill(np.nan)
    #iterate through the movies_df
    for index, row in rdf.iterrows():
        new_movie_id = get_new_movie_id_from_old(row['movieId'])
        user_id = int(row['userId'])
        full_ratings_mat[user_id][new_movie_id] = row['rating']
    full_ratings_mat=full_ratings_mat[1:]
        
    return full_ratings_mat

#the mode can be "mean" i.e fill the empty values with mean of the rows (users in this case)
#or fill the empty values with zeros...in that case the mode is "zero"
def fill_sparse_matrix(mat, mode):
    num_users = mat.shape[0]
    num_movies = mat.shape[1]
    
    mat_copy = np.copy(mat)
    
    if mode!="mean" and mode!="zero":
        raise ValueError("Invalid Mode...mode should be either mean or zero")
    else:
        for u_id in range(num_users):
            u_mean = np.nanmean(mat_copy[u_id])
            for m_id in range(num_movies):
                if np.isnan(mat_copy[u_id][m_id])==True:
                    if mode=="mean":
                        mat_copy[u_id][m_id] = u_mean
                    else:
                        mat_copy[u_id][m_id] = 0.0
    return mat_copy

def get_observed_pairs_list(ratings_matrix):
    res = []
    
    num_users = ratings_matrix.shape[0]
    num_movies = ratings_matrix.shape[1]
    
    for u_id in range(num_users):
        for m_id in range(num_movies):
            if np.isnan(ratings_matrix[u_id][m_id])==False:
                res.append([u_id, m_id])
                
    return res


def nan_frobenius(mat_2d):
    num_users = mat_2d.shape[0]
    num_movies = mat_2d.shape[1]
    res =0.0
    for user_num in range(num_users):
        for movie_num in range(num_movies):
            res = res + np.square(mat_2d[user_num][movie_num])
            
    return res

In [ ]:
NUM_MOVIES = len(mdf)
NUM_USERS = 610

In [ ]:
ratings_matrix = create_the_rating_matrix(mdf, NUM_MOVIES, NUM_USERS)
observed_pairs_list = get_observed_pairs_list(ratings_matrix)

In [ ]:
# filled_ratings_matrix_mean
filled_ratings_matrix_mean = fill_sparse_matrix(ratings_matrix, mode="mean")
# filled_ratings_matrix_mean

In [ ]:
# filled_ratings_matrix_zero
filled_ratings_matrix_zero = fill_sparse_matrix(ratings_matrix, mode="zero")
# filled_ratings_matrix_zero

In [ ]:
ratings_matrix.shape
"""
Note-->users and movies are zero indexed 
"""

In [ ]:
"""
Q1 begins here
"""
U_final =0
V_final = 0

def unconst_fact_batch(ratings_matrix, filled_ratings_matrix, K, lr=1.5e-06, load_from_file=True):
    num_users = ratings_matrix.shape[0]
    num_movies = ratings_matrix.shape[1]
    
    #init U and V
    if load_from_file==False:
        U = np.random.rand(num_users, K)
        V = np.random.rand(num_movies, K)
    else:
        U = np.load('unconst_fact_batch_U.npy')
        V = np.load('unconst_fact_batch_V.npy')
    
    iters = 0
    
    while True:
        curr_mat = np.matmul(U, V.T)
        E = filled_ratings_matrix - curr_mat
        #calculate U updates
        U_update_mat = U + lr*np.matmul(E, V)
        V_update_mat = V + lr*np.matmul(E.T, U)
        
        U = U_update_mat
        V = V_update_mat
        
        err_old = np.linalg.norm(E)
        E_new = filled_ratings_matrix - np.matmul(U, V.T)
        err_new = np.linalg.norm(E_new)
        
        U_final = U
        V_final = V
        
        if iters%10==0:
            print("Iters--> {}; Error--> {}".format(iters, err_old, err_new))
            #save the result of the unconst_fact_batch to file
            np.save('unconst_fact_batch_U.npy', U_final)
            np.save('unconst_fact_batch_V.npy', V_final)
        
        iters = iters+1
        
        
        if abs(err_old-err_new)<=1e-03:
            break
         

In [ ]:
unconst_fact_batch(ratings_matrix, filled_ratings_matrix_mean, 100)

In [ ]:
def unconst_fact_stoch(ratings_matrix, filled_ratings_matrix, observed_pairs_list, K, lr=1.5e-06, load_from_file=True):
    num_users = ratings_matrix.shape[0]
    num_movies = ratings_matrix.shape[1]
    
    rand_pairs_list = observed_pairs_list
    
    #init U and V
    if load_from_file==False:
        U = np.random.rand(num_users, K)
        V = np.random.rand(num_movies, K)
    else:
        U = np.load('unconst_fact_stoch_U.npy')
        V = np.load('unconst_fact_stoch_V.npy')
    
    error_diff = 1000
    
    iters = 0
    
    while True: #condition for convergence
        E_old = ratings_matrix - np.matmul(U, V.T)
        E_old = np.where(np.isnan(E_old)==True, 0, E_old)
        
        err_old = np.linalg.norm(E_old)
        
        random.shuffle(rand_pairs_list)
        
        E_new = np.empty(E_old.shape)
        
        for u_i_pair in rand_pairs_list:
            u_id = u_i_pair[0]
            m_id = u_i_pair[1]
            
            eij = ratings_matrix[u_id][m_id] - np.dot(U[u_id], V[m_id])
            
            E_new[u_id][m_id] = eij
            
            ui_update = U[u_id] + lr*eij*V[m_id]
            vi_update = V[m_id] + lr*eij*U[u_id]
            
            U[u_id] = ui_update
            V[m_id] = vi_update
            
            
            #calculate the new error
        E_new = np.where(np.isnan(E_new)==True, 0, E_new)
        
        err_new = np.linalg.norm(E_new)
        
        if abs(err_old-err_new)<1e-03: break
        
        if iters%10==0:
            print("iters-->{}; old_error-->{}; new_error-->{}".format(iters, err_old, err_new))
            np.save('unconst_fact_stoch_U.npy', U_final)
            np.save('unconst_fact_stoch_V.npy', V_final)
            
        iters = iters+1
        

In [ ]:
unconst_fact_stoch(ratings_matrix, filled_ratings_matrix_mean, observed_pairs_list, 100, lr=1.5e-06, load_from_file=False)

In [ ]:
"""
Q1 Unconstrained matrix factorization with stochastic and regularisation 
"""
U_final =0
V_final = 0

def unconst_fact_stoch_reg(ratings_matrix, filled_ratings_matrix, observed_pairs_list, K, lr=1.5e-06, lam = 1, load_from_file=True):
    num_users = ratings_matrix.shape[0]
    num_movies = ratings_matrix.shape[1]
    
    rand_pairs_list = observed_pairs_list
    
    #init U and V
    if load_from_file==False:
        U = np.random.rand(num_users, K)
        V = np.random.rand(num_movies, K)
    else:
        U = np.load('unconst_fact_stoch_reg_U'+str(lam)+'.npy')
        V = np.load('unconst_fact_stoch_reg_V'+str(lam)+'.npy')
    
    error_diff = 1000
    
    iters = 0
    
    while True: #condition for convergence
        E_old = ratings_matrix - np.matmul(U, V.T)
        E_old = np.where(np.isnan(E_old)==True, 0, E_old)
        
        err_old = np.linalg.norm(E_old)
        
        random.shuffle(rand_pairs_list)
        
        E_new = np.empty(E_old.shape)
        
        for u_i_pair in rand_pairs_list:
            u_id = u_i_pair[0]
            m_id = u_i_pair[1]
            
            eij = ratings_matrix[u_id][m_id] - np.dot(U[u_id], V[m_id])
            
            E_new[u_id][m_id] = eij
            
            ui_update = U[u_id] + lr*eij*V[m_id] - lr*lam*U[u_id]
            vi_update = V[m_id] + lr*eij*U[u_id] - lr*lam*V[m_id]
            
            U[u_id] = ui_update
            V[m_id] = vi_update
            
            
            #calculate the new error
        E_new = np.where(np.isnan(E_new)==True, 0, E_new)
        
        err_new = np.linalg.norm(E_new)
        
        if abs(err_old-err_new)<1e-03: break
        
        if iters%10==0:
            print("iters-->{}; old_error-->{}; new_error-->{}".format(iters, err_old, err_new))
            np.save('unconst_fact_stoch_reg_U'+str(lam)+'.npy', U_final)
            np.save('unconst_fact_stoch_reg_V'+str(lam)+'.npy', V_final)
            
        iters = iters+1
    #return the reconstructed matrices
    return U, V
        
def get_matrix_accu_validation(U, V, validation_rating_matrix):
    #reconstruct the matrix in one shot
    R = np.matmul(U*V.T)
    
    #now get the error matrix
    E = R-validation_rating_matrix
    E = np.where(np.isnan(E)==True, 0, E)
    #fill the empty vals with zero
    return np.linalg.norm(E)
        

In [ ]:
def split_the_dset(input_rating_matrix, train_ratio=80.0, mean_center=False):
    #sample from a uniform distribution to prepare a 
    np.random.seed(0) ;
    mask = np.random.uniform(low=0.0, high=100.0, size=input_rating_matrix.shape)
    #set the curoff at train ratio
    mask = mask<=train_ratio
    
    train_rating_matrix = np.empty(shape=input_rating_matrix.shape)
    train_rating_matrix.fill(np.nan)

    test_rating_matrix = np.empty(shape=input_rating_matrix.shape)
    test_rating_matrix.fill(np.nan)

    #loop through the elments of the input training matrix
    for user in range(input_rating_matrix.shape[0]):
        for item in range(input_rating_matrix.shape[1]):
            if mask[user][item]==True and input_rating_matrix[user][item]!=np.nan:
                train_rating_matrix[user][item] = input_rating_matrix[user][item]
            elif mask[user][item]==False and input_rating_matrix[user][item]!=np.nan:
                test_rating_matrix[user][item] = input_rating_matrix[user][item]

    return train_rating_matrix, test_rating_matrix

In [ ]:
#split the dset for cross validation purposes
train_rating_matrix, validation_rating_matrix =  split_the_dset(ratings_matrix)

In [ ]:
lam_vals = []
validation_accus = []
K=100
for lam in range(1, 50, 5):
    print("Lambda = {};".format(lam))
    Ul, Vl = unconst_fact_stoch_reg(ratings_matrix, filled_ratings_matrix_mean, observed_pairs_list, K, lr=1.5e-06, lam=lam,  load_from_file=False)
    lam_vals.append(lam)
    validation_accus.append(get_matrix_accu_validation(Ul, Vl, validation_ratings_matrix))
    
    plt.xlabel('Lambda Values')
    plt.ylabel('Error')
    plt.scatter(lam_vals, validation_accus)
    plt.savefig('1b.png')

In [ ]:
"""
Q2. begins here
"""
#convert a 1d matrix of singluar values returned by the svd function to a 2d square matrix
def convert_singluar_to_diagonal(mat):
    sz = mat.shape[0]
    res = np.zeros(shape=(sz, sz))
    for i in range(sz):
        res[i][i] = mat[i]
    return res

#perform svd
def svd(ratings_matrix, filled_ratings_matrix_mean, k=100):
    #mean center the ratings_matrix
    num_users = ratings_matrix.shape[0]
    num_movies = ratings_matrix.shape[1]
    
    #fill missing entries with zero i.e the mean
    rm = filled_ratings_matrix_mean
    
    iters = 0
    
    #now do the iterations
    while True:
        #step 1 
        u,s,v = np.linalg.svd(rm)
        s = np.reshape(s, (s.shape[0], 1))
#         print(u.shape, s.shape, v.shape)
#         stemp = s[:k]
#         print(stemp.shape)
        s = convert_singluar_to_diagonal(s)
        
        #get the top-k eigenvals etc.
        u = u[:, :k]
        v = v[:, :k]
        s = s[:k, :k]
#         print(u.shape, s.shape, v.shape)
        #readjust missing vals of orig ratings matrix
        pred_rm = np.matmul(np.matmul(u, s), v.T)
        #step 2
        #readjust the missing vals 
        for u_id in range(num_users):
            for m_id in range(num_movies):
                if np.isnan(ratings_matrix[u_id][m_id])==True:
                    rm[u_id][m_id] = pred_rm[u_id][m_id]
                    
        if iters>100: return u,s,v
        else:
            #find the error and print it
            E = pred_rm - ratings_matrix
            E = np.where(np.isnan(E)==True, 0, E)
            print("Iters--> {}; Error--> {}".format(iters, np.linalg.norm(E)))
            iters = iters+1
    

In [ ]:
u,s,v = svd(ratings_matrix, filled_ratings_matrix_mean)

In [ ]:
"""
Q3. begins here
"""

def get_non_negative_mat_factorization_reg(ratings_matrix, k=100, lam1=20, lam2=20, lr=1.5e-06, eps=1e-09, load_from_file=True):
    num_movies = ratings_matrix.shape[1]
    num_users = ratings_matrix.shape[0]
    #init U and V with [0,1] randomly
    R = fill_sparse_matrix(ratings_matrix, mode = 'zero')
    if load_from_file==False:
        U = np.random.random_sample((num_users, k))
        V = np.random.random_sample((num_movies, k))
    else:
        U = np.load('non_negative_mat_factorization_reg_U'+str(lam1)+'.npy')
        V = np.load('non_negative_mat_factorization_reg_V'+str(lam1)+'.npy')
    
    iters = 0
    
    err_old=0
    
    while True:
        rv = np.matmul(R, V)
        rtu = np.matmul(R.T, U)
        uvtv = np.matmul(U, np.matmul(V.T, V))
        vutu = np.matmul(V, np.matmul(U.T, U))
        
        u_update = np.empty(U.shape)
        u_update = np.multiply(np.divide(rv-lam1*U, uvtv+eps), U)
        u_update = np.where(u_update<0, 0, u_update)
        
        v_update = np.empty(V.shape)
        v_update = np.multiply(np.divide(rtu-lam2*V, vutu+eps), V)
        v_update = np.where(v_update<0, 0, v_update)
        
#         for u_id in range(num_users):
#             for j in range(k):
#                 #set the values according to the update rules 
#                 u_update = np.copy(U)
#                 u_update[u_id][j] = max(0, ((rv[u_id][j]-lam1*U[u_id][j])/(uvtv[u_id][j]+eps))*U[u_id][j])
                                           
#         for m_id in range(num_movies):
#             for j in range(k):
#                 v_update = np.copy(V)
#                 v_update[m_id][j] = max(0, ((rtu[m_id][j]-lam2*V[m_id][j])/(vutu[m_id][j]+eps))*V[m_id][j])

                                           
        U = u_update
        V = v_update
        
        E_new = ratings_matrix-np.matmul(U, V.T)
        E_new = np.where(np.isnan(E_new), 0, E_new)
        err_new = np.linalg.norm(E_new)
        
        if iters%10==0:
            print("Iters-->{}; Error-->{};".format(iters, err_new))
            np.save('non_negative_mat_factorization_reg_U'+str(lam1)+'.npy', U_final)
            np.save('non_negative_mat_factorization_reg_U'+str(lam2)+'.npy', V_final)
            
        if abs(err_new-err_old)<=1e-03:
            return U, V
            break
        
        err_old = err_new
        
        iters = iters+1

In [ ]:
u,v = get_non_negative_mat_factorization_reg(ratings_matrix, load_from_file=True)